<a href="https://colab.research.google.com/github/byuccl/digital_design_colab/blob/main/Lab3_Seven_Segment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="right">
<img src="https://raw.githubusercontent.com/byuccl/digital_design_colab/master/Dataflow/media/BYU.svg"
width="500" height="" style="display: block; margin: 0 auto" />
</p>


<h1>
<summary>
<h1 align="center">
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<b>$\color{midnightBlue}{\underline{\textbf{Seven Segment Display}}}$<bol><p align="right">
</summary>
</h1>

## **Setup**
Click the Play Button. No need to expand. Do not touch this. 
<details><summary>Packages to be downloaded</summary>
Things it needs to install:  <br>

 *    Verilator- the simulator  <br>
 *    PyGithub - Helps with simulation    <br>
 *    VCD - Generates waveforms from `.vcd` files  <br>
 *    Widgets - Each widget must first be generated then will be loaded when played
</details>

In [ ]:
#@title Install VCD { form-width: "1795px" }
%%bash
git clone https://github.com/yne/vcd.git
make -C /content/vcd
cd /content/vcd
make install

make: Entering directory '/content/vcd'
cc  vcd.c -o vcd
make: Leaving directory '/content/vcd'
install -v -D vcd /usr/bin/vcd
'vcd' -> '/usr/bin/vcd'


Cloning into 'vcd'...


In [ ]:
#@title Import Files from Repo
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout, jslink, IntText, IntSlider
import requests
def import_text(text):
  url = 'https://raw.githubusercontent.com/byuccl/digital_design_colab/master/SevenSegment/files/%s' % text
  resp = requests.get(url)
  with open(text, 'wb') as f:
    f.write(resp.content)
def import_text_bin(text):
  url = 'https://raw.githubusercontent.com/byuccl/digital_design_colab/master/bin/%s' % text
  resp = requests.get(url)
  with open(text, 'wb') as f:
    f.write(resp.content)
# import_text("tb_dataflow_sv.cpp")
# import_text("dataflow_sv3.cpp")
# import_text("tb_func2.cpp")
import_text_bin("quiz_dict01.py")
import_text_bin("tt_widget.py")
import_text_bin("frquestion.py")

import tt_widget as tt
import quiz_dict01 as quiz_bot 
from frquestion import *
def quiz_func(a):
  quiz_bot.quiz_func(a)

!git clone https://github.com/westonMS/FPGA_Colablab.git

Cloning into 'FPGA_Colablab'...
remote: Enumerating objects: 999, done.
remote: Counting objects: 100% (296/296), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 999 (delta 166), reused 221 (delta 123), pack-reused 703
Receiving objects: 100% (999/999), 2.10 MiB | 19.95 MiB/s, done.
Resolving deltas: 100% (480/480), done.


In [ ]:
#@title Nelson's Simulation Widget { form-width: "50px" }
import os
import re
import sys
import subprocess
from collections import defaultdict, namedtuple
import subprocess
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout, jslink, IntText, IntSlider
from google.colab import output
output.enable_custom_widget_manager()

!mkdir /content/tmp_code
def vcd():
  !vcd < /content/tmp_code/logs/vlt_dump.vcd > /content/vcd.txt
  !sed -i '1,10d'  /content/vcd.txt 
  !cat /content/vcd.txt


VerilogIO = namedtuple("VerilogIO", "dir nam wid")
'''
        Using a namedtuple (sort of like a C-struct) to hold info on each Verilog IO.
        This is very similar to using a class with only data members.
          The advantages are: (1) the fields are not change-able after creation
          (which is what I want), and (2) probably more efficient than using a class.
        This is used by both generate() and processVmoduleHFile() so it is declared in global scope.
'''


def getModuleName(textSourceCode):
  match_name = re.search(r"module\s{1,}\w{1,}\s{0,}#{0,}\s{0,}\(",textSourceCode.value)
  if match_name is None:
      return None
  mname = match_name.group().split(' ')[1]
  mname = mname.split('(')[0]
  mname = mname.split('#')[0]
  return mname
def writeJson(d, fname='/tmp/json.json', ind=2):
    ''' Write an object to a file with indenting '''
    import json
    with open(fname, encoding='utf-8', mode='w') as f:
        json.dump(d, f, indent=ind)
def cd(folder):
    ''' My wrapper for os.chdir() so I can print it desired. '''
    if folder:
        os.chdir(folder)
    #print("##### Changing to " + os.getcwd() + "#####")

def getNextLine(f):
    ''' Read file f until you get a non-comment and a non-blank link.
        The returned line will be stripped of whitespace (including newline)
    '''
    while True:
        myLine = f.readline().strip()
        if not myLine.startswith('//') and len(myLine) > 0:
            break
    return myLine

def processSTMHeader(tBench):
    '''
        Read STM file and pull out the clock definition and the list of inputs.
        Note that for combinational circuits we are expecting a clock line
          consisting of 'comb' (which is ignored)
    '''
    with open(tBench, encoding='utf-8', mode='r') as f:
        stmClock = None
        tmp = getNextLine(f)
        if tmp != 'comb':
            stmClock = tmp
        inputs = getNextLine(f).split()

    # Verify that a signal doesn't appear twice in the list (this is a user-created file after all)
    for elmt in inputs:
        assert inputs.count(elmt) == 1, "Cannot have duplicate names in STM file input list. ({} appears more than once)".format(elmt)

    return (stmClock, inputs)

def processVmoduleHFile(fname, stmClock, stmInputs):
    '''
        Process the verilator-produced Vmodule.h file and pull out
        the IO definitions.  Pack them into the verilogIOs structure,
        which is a list of lists consisting of [dir, name, size].
        Along the way see if you can match any of the signals as
        corresponding to the clock defined in the stimulus file.
        Finally, make sure that all verilog signals are in the stm file
          and the other way around.
    '''

    verilogIOs = []
    with open(fname, encoding='utf-8', mode='r') as f:
        verilogClock = None
        for line in f:
            line = line.strip()
            if line.startswith('// LOCAL VARIABLES'): # Can skip everything after this line
                break
            match = re.match(r'^VL_(IN|OUT|SIG)(\d*)\(([^,]*),(\d+),(\d+)', line)
            if match is not None:
                varDir, varName, varSize = [match.groups()[0], match.groups()[2], int(match.groups()[3]) - int(match.groups()[4]) +1]
                verilogIOs.append(VerilogIO(varDir, varName, varSize))
                # If input, check that it is in STM file
                if varName == stmClock:
                    verilogClock = varName   # Found matching clock names between STM and Verilog header

    # Check to make sure that every stm input is in the verilog file
    #   and the other way around.
    for si in stmInputs:
        assert si in [f.nam for f in verilogIOs if f.dir=='IN'], \
            "ERROR: input '{}' in STM file but not in Verilog design".format(si)
    for vi in [f.nam for f in verilogIOs if f.dir=='IN']:
        assert vi == stmClock or vi in stmInputs, \
            "ERROR: input '{}' in Verilog design but not in STM file".format(vi)

    # Make sure that if a clock was in the STM file, it was found in the Verilog
    if stmClock != None:
        assert verilogClock != None, "There was a clock defined in the STM file but no corresponding clock in the Verilog file."
    return (verilogClock, verilogIOs)
from string import Template
def generate(designFolder, verilogFile, tBenchFile,
             draw = True, wall = True, compress = True, Hex = True, verbose = False,
             pythonPath = '', processClockWaveform = True,
             hscale = 1, nameOrder = ''):
    """
    Generate a C++ object of verilog/systemverilog code using verilator, toggle inputs on said C++ object,
    and portray resulting waveforms of the signals. Comments in the stm file are supported and are indicated with //
    Comments must start at the beginning of the line and are expected to take up the whole line
    Keyword Arguements:
    designFolder -- str - relative path of the directory, where the .v and .stm files are
    verilogFile -- str - the top level HDL module
    tBenchFile -- str - the name of a .stm file of input stimulus.
    draw -- bool - whether or not to display the waveform
    wall -- bool whether or not to check for verilator warnings
    compress -- bool whether or not to compress the c++ object made by verilator. (Compression effectively
      eliminates the ability to view most internal signals  (deprecated)
    Hex -- bool - whether to display waveform values in hexadecimal
    verbose -- bool - control verbose mode
    pythonPath -- str - path to directory where MaverickSimulator project can be found. If left empty, the typical
                        PYTHONPATH/site-packages will be assumed
    processClockWaveform -- bool -  By default, will attempt to convert the clock waveform to one with arrows.
                                    Setting this to False will prevent this
    hscale -- int - Will adjust the width of the ticks on the waveform diagram. This arguement is passed directly
                    to executeSimulation()
    """
    clock = None    # The name of the clock to the design (if any)
    homeDir = os.getcwd()
    try:
        fileNameRoot = os.path.splitext(verilogFile)[0]
        if not pythonPath:
            dirs = sys.path
            for dir in dirs:
                if (dir.find('site-packages') != -1 ):
                    #TODO add check to make sure path is valid
                    pythonPath = dir + '/MaverickSimulator/src'
                    break
        # Adjust commandline args for verilator
        w = ' -Wall' if wall else ' '
        c = ' ' if compress else ' --public '
        wv = ' --trace ' if draw else '--trace'

        cd(designFolder)

        # Step 1: run the verilator program
        v = subprocess.Popen(r'verilator -cc'+ w + c + wv + verilogFile + " -CFLAGS '-std=c++11' --exe sim___.cpp",    #create verilator object and link to .cpp
                             stdout = subprocess.PIPE, stderr = subprocess.STDOUT, shell = True, universal_newlines=True)
        verbose = 0
        if verbose:
            print('Running verilator...')
            print('verilator -cc'+ w + c + wv + verilogFile + " -CFLAGS '-std=c++11' --exe sim___.cpp")
            output = v.stdout.read().strip()
            if len(output) > 0:
                print('Results of running verilator: ' + output)
            if output.find('%Error: Command Failed') != -1:
                return 1
                sys.exit("Exiting due to verilator errors")
        else:
            output = v.stdout.read().strip()
            if output.find('%Error: Command Failed') != -1:
                print(output)
                os.chdir(homeDir)
                return 1
                sys.exit("Exiting due to verilator errors")

        # Step 2: Parse STM and Verilog files
        # Open .stm file and get clock and list of inputs from the  first two uncommented lines
        stmClock, stmInputs = processSTMHeader(tBenchFile)

        # Read the IO definitions from the Vmodule.h file
        # Add inputs, outputs, and local signals to data structure verilogIOs
        verilogClock, verilogIOs = processVmoduleHFile('obj_dir/V'+fileNameRoot+'.h', stmClock, stmInputs)

        # If verilogClock has been set that means
        # it matched up with the one from the STM file. Otherwise, it did not.
        # The rest of the code will use clock to know this.
        if verilogClock != None:
            clock = verilogClock

        # Step 3:Write the Testbench__lh file
        ios = ''
        sg = ''
        # Build list of IO print stmts for printIOs()
        for nam, wid in [(f.nam, f.wid) for f in verilogIOs if f.dir!='SIG']:
            ios += '    std::cout << " {0} is " << std::bitset<{1}>(top->{0}) <<  std::endl;\n'.format(nam, wid)
        # If needed, build list of IO print stmts for printInternalOs()
        if not compress:
            for nam, wid in [(f.nam, f.wid) for f in verilogIOs if f.dir=='SIG']:
                varName = nam.replace('v__DOT__', '').replace('__DOT__', ' ')
                sg += '   std::cout << "{} is " << std::bitset<{}>(top->{}) << std::endl;\n'.format(varName, wid, nam)
        # Now actually write the testBench file into the design directory
        with open('TestBench___.h', encoding='utf-8', mode='w') as tb: #header file for interacting with verilator object
            with open("/content/tmp_code/TBtemplate.txt", 'r') as tbTemplate:
                t = Template(tbTemplate.read())
                t = t.substitute(
                    MODULE=fileNameRoot,
                    CLKDEF = '\n#define CLK top->'+clock if clock else '',
                    IOPRINTS = ios,
                    INTERNALIOPRINTS = sg
                    )
                tb.write(t)

        # Step 4: write the main simulation file
        # Collect IO print code snippets (using simTemplate2.txt as pattern)
        ios = ''
        for inputName in stmInputs:
            for inputWidth in [f.wid for f in verilogIOs if f.nam==inputName]:
                with open("/content/tmp_code/simTemplate2.txt", 'r') as simTemplate2:
                    t = Template(simTemplate2.read())
                    t = t.substitute(SIGNAME = inputName, SIGWIDTH = inputWidth)
                    ios += t

        # Now write the main simulation file for compilation (using simTemplate.txt as pattern)
        with open('sim___.cpp', encoding='utf-8', mode='w') as sim:
            with open("/content/tmp_code/simTemplate.txt",
                    encoding='utf-8', mode='r') as simTemplate:  #now write main.cpp file
                t = Template(simTemplate.read())
                t = t.substitute(
                            NUMINS = len(stmInputs),
                            READINPUTSCODE = ios,
                            HASCLOCK = (0 if clock is None else 1)
                            )
                sim.write(t)

        # Step 5: Compile the resulting program
        print("Compiling")
        if verbose:
            print('make -C obj_dir -j -f V'+fileNameRoot+'.mk V'+fileNameRoot+'; cd obj_dir')
            v = subprocess.Popen(r'make -C obj_dir -j -f V'+fileNameRoot+'.mk V'+fileNameRoot+'; cd obj_dir',
                                 stdout = subprocess.PIPE, stderr = subprocess.STDOUT, shell = True, universal_newlines=True)
            output = v.stdout.read().strip()
            if len(output) > 0:  print('   Results of compiling: ' + output)
        else:
            v = subprocess.Popen(r'make -C obj_dir -j -f V'+fileNameRoot+'.mk V'+fileNameRoot+'; cd /content/tmp_code/obj_dir',    #call makefile to g++ compile appropriate files
                                 stdout = subprocess.PIPE, stderr = subprocess.STDOUT, shell = True, universal_newlines=True)

        output = v.stdout.read().strip()
        # Step 6: Execute it
        # if not nameOrder:
        #   names = [f.nam for f in verilogIOs if f.dir != 'SIG']
        #   writeJson(names, fname=fileNameRoot+'.nmo')
        #   nameOrder = fileNameRoot+'.nmo'
        # cd(homeDir)
        # ret = executeSimulation(designFolder, verilogFile, tBenchFile, draw,
        #                     compress, Hex, verbose, clock, nameOrder,
        #                     processClockWaveform, hscale, pythonPath)
        # if draw is False:
        #     return ret
        subprocess.run(["/content/tmp_code/obj_dir/V" +fileNameRoot, "/content/tmp_code" + fileNameRoot + ".stm", "--trace" ])
        # !./tmp_code/obj_dir/VbehavLoadableReg /content/tmp_code/temp.stm --trace
      # !vcd < /content/logs/vlt_dump.vcd
        subprocess.run(["vcd", "<", "/content/logs/vlt_dump.vcd"])
    except Exception as e:
        os.chdir(homeDir)
        print(str(e))
def showWaveformClicked(self):
    sc = cellDict[self]
    assert sc is not None, "Internal error with cellDict"

    if submitCodes(sc.interpreterHomeDir, sc.textSourceCode, sc.textStimulus) == False:
        return
    srcFileName = getModuleName(sc.textSourceCode)
    arrow = sc.chkArrow.value
    hex = True if sc.selectHexBin.value is 'Hex' else False
    assert os.path.exists(sc.interpreterHomeDir+'/tmp_code/'+srcFileName+'.sv'), "Please submit the codes before showing waveform: {} {}".format(sc.interpreterHomeDir, srcFileName)
    generate(sc.interpreterHomeDir+'/tmp_code', srcFileName+'.sv', srcFileName+'.stm', compress=True, Hex=hex, verbose=True, processClockWaveform=arrow)
    if sc.showCode.value:
        print("------------------------------------------------------------------------------------------")
        print("Source code for the simulation above:")
        print("------------------------------------------------------------------------------------------")
        print(sc.textSourceCode.value)
        print("------------------------------------------------------------------------------------------")

    print("###########################################################################################")
    x = subprocess.run(["/content/tmp_code/obj_dir/V" +srcFileName, "/content/tmp_code/" + srcFileName + ".stm", "--trace" ], stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    print(x.stderr + x.stdout)
    # x = subprocess.run(["vcd", "< /content/tmp_code/logs/vlt_dump.vcd" ], stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    # print(str(x.stderr) + str(x.stdout))
    # # os.system("vcd < /content/tmp_code/logs/vlt_dump.vcd" )
    vcd()
class SimCell:
  def __init__(self, textSourceCode, textStimulus, chkArrow, selectHexBin, interpreterHomeDir, showCode, contents):
      self.textSourceCode = textSourceCode
      self.textStimulus = textStimulus
      self.chkArrow = chkArrow
      self.selectHexBin = selectHexBin
      self.interpreterHomeDir = "/content"
      self.showCode = showCode
      self.contents = contents
def readContents(contents):
    if contents is None:
        code = ''
        stim = ''
    else:
        print(contents)
        contents = "/content/" + contents 
        with open(contents + ".sv") as f:
            code = f.read()
        with open(contents + ".stm") as f:
            stim = f.read()
        # with open(JTEXT+'/pgms/' + contents[:-1] + "tm") as f:
        #     stim = f.read()
    return [code, stim]

cellDict = dict()

def CreateWidgets(contents, ht="125px", wid="500px", stht='125px', stwid='500px'):
    code, stim = readContents(contents)

    srcCodeWidget = widgets.Textarea(
                            disabled=False,
                            value = code,
                            layout=widgets.Layout(height=ht,width=wid),
                            )
    stimulusWidget = widgets.Textarea(
                            value=stim,
                            disabled=False,
                            layout=widgets.Layout(height=stht, width=stwid),
                            )

    selectHexBin = widgets.Dropdown(options=['Binary','Hex'],
                                 value='Hex',
                                 description='',
                                 disabled=False,
                                 )
    chkArrow = widgets.Checkbox(description='Show the clock arrow', value=True, indent=False)
    showCode = widgets.Checkbox(description='Show source code below waveform', value=False, indent=False)
    #butt_submit = widgets.Button(description='Save code')
    butt_show_waveform = widgets.Button(description='Run simulation')
    butt_refresh_contents = widgets.Button(description='Restore code')
    butt_save_code = widgets.Button(description = "Save code")
    return srcCodeWidget, stimulusWidget, selectHexBin, chkArrow, showCode, butt_show_waveform, butt_refresh_contents, butt_save_code
def submitCodes(interpreterHomeDir, textSourceCode, textStimulus):
    if textSourceCode.value is '' or textStimulus.value is '':
        print("The source code and stimulus cannot be empty! Please input again")
        print("###########################################################################################")
        return False
    else:
        with open(interpreterHomeDir + "/tmp_code/" + getModuleName(textSourceCode) + ".sv", 'w') as f:
            f.write(textSourceCode.value)
        with open(interpreterHomeDir + "/tmp_code/" + getModuleName(textSourceCode) + ".stm", 'w') as f:
            f.write(textStimulus.value)
    return True

def refreshContentsClicked(self):
    sc = cellDict[self]
    assert sc is not None, "Internal error with cellDict"
    code, stim = readContents(sc.contents)
    sc.textSourceCode.value = code
    sc.textStimulus.value = stim

def overwriteSavedCode(self):
    sc = cellDict[self]
    print(sc.contents)
    file_location = "/content/" + sc.contents
    with open(file_location + ".sv", "w") as f:
      f.write(sc.textSourceCode.value)
    with open(file_location+".stm", "w") as f:
      f.write(sc.textStimulus.value)

def createSimulationWorkSpace(initialContents = None, ht='125px', wid='500px', stht='125px', stwid='500px'):

    interpreterHomeDir = os.getcwd()
    code_file_path = interpreterHomeDir+'/tmp_code'
    if not os.path.exists(code_file_path):
        os.mkdir(code_file_path)

    # Create widgets for this part
    textSourceCode, textStimulus, selectHexBin, chkArrow, showCode, btnShowWaveform, btnRestoreCode, btnSaveCode = CreateWidgets(initialContents, ht, wid, stht, stwid)
    #print(textSourceCode.value)
    #print(textStimulus.value)
    # Register callbacks for when clicking the button
    btnShowWaveform.on_click(showWaveformClicked)
    btnRestoreCode.on_click(refreshContentsClicked)
    btnSaveCode.on_click(overwriteSavedCode)
    # Register this cell's info
    sc = SimCell(textSourceCode, textStimulus, chkArrow, selectHexBin, interpreterHomeDir, showCode, initialContents)
    cellDict[btnShowWaveform] = sc
    cellDict[btnRestoreCode] = sc
    cellDict[btnSaveCode] = sc


    # Draw cell widgets
    leftPart = widgets.VBox([
                        widgets.VBox([
                                widgets.Label('Source code area:'),
                                textSourceCode
                                ]),
                        widgets.VBox([
                                widgets.Label('Simulation code area:'),
                                textStimulus
                                ])
                        ])
    rightPart = widgets.VBox([
                                widgets.Label(''),
                                widgets.Label(''),
                                widgets.Label(''),
                                #widgets.Label(''),
                                widgets.Label(''),
                                showCode,
                                #widgets.Label(''),
                                btnShowWaveform,
                                #widgets.Label(''),
                                btnSaveCode,
                                widgets.Label(''),
                                widgets.Label(''),
                                widgets.Label(''),
                                widgets.Label(''),
                                widgets.Label(''),
                                btnRestoreCode
                            ])

    # Show widgets
    display(widgets.HBox([leftPart,rightPart]))


In [ ]:
#@title Import Simulation Files
cd("/content/tmp_code")

import_text_bin("TBtemplate.txt")
import_text_bin("simTemplate.txt")
import_text_bin("simTemplate2.txt")
import_text("seven_segment.sv")
import_text("seven_segment.stm")
import_text("seven_segment_top.sv")
import_text("seven_segment_top.stm")
# import_text("arithmetic_top.sv")
# import_text("arithmetic_top.stm")
cd("/content")

In [ ]:
#@title Grid Creation
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout, jslink, IntText, IntSlider

# function of creating button widget
def create_expanded_button(description, button_style, width="100px"):
    return Button(
        description=description,
        button_style=button_style,
        layout=Layout(height="auto", width=width),
    )


# creates variable grid size
def create_grid(num_input, input_string, func):
    """
    - num_input = number of inputs in the function
    - input_string = a string of all the default values and input names that are passed in (ex: AB000011011)
    """
    num_row = (2**num_input) + 1
    num_col = num_input + 2
    grid = GridspecLayout(num_row, num_col, width=str(num_col * 117.5) + "px")

    # creates the default values for the different input combos of the truth table
    for i in range(0, num_row):
        for j in range(0, num_input):
            val = str(input_string[0])
            grid[i, j] = create_expanded_button(val, "info")
            input_string = input_string[1:]

    # creates the click to check button as well as the userinput section of the table
    for i in range(1, num_row):
        grid[i, num_input] = widgets.BoundedIntText(
            min=0, max=1, layout=Layout(height="auto", width="100px")
        )
        grid[0, num_col - 1] = create_expanded_button(
            "Click to Check", "warning", width="150px"
        )

    # creates the output section of the truth table to know if you're right or not
    for i in range(1, num_row):
        grid[i, num_col - 1] = create_expanded_button(" ", "warning", width="150px")

    grid[0, num_input] = create_expanded_button(func, "info")
    return grid


# checks if the user input is correct
def CheckAnswer(grid, num_inputs, input):
    for i in range (0,len(input)):
        if grid[i + 1, num_inputs].value == int(input[i]):
            grid[i + 1, num_inputs + 1].button_style = "Success"
            grid[i + 1, num_inputs + 1].description = "Correct!"
        else:
            grid[i + 1, num_inputs + 1].button_style = "Danger"
            grid[i + 1, num_inputs + 1].description = "Wrong! Submit again"


# AND gate truth table creation
grid1_1 = create_grid(2, 'AB00011011', 'F=AB')

# OR gate truth table creation
grid1_2 = create_grid(2, 'AB00011011', 'F=A+B')

# NOT gate truth table creation
grid1_3 = create_grid(1, "A01", '!A')

# NOR gate truth table creation
grid1_4 = create_grid(2, 'AB00011011', 'F=NOR')

# NAND gate truth table creatione
grid1_5 = create_grid(2, 'AB00011011', 'F=NAND')

# XOR gate truth table creation
grid1_6 = create_grid(2, 'AB00011011', 'F=A^B')

# XNOR gat truth table creatione
grid1_7 = create_grid(2, 'AB00011011', 'F=~(A^B')

# THIS CAN PROBABLY BE PLACED IN THE CREATE FUNCTION WITH F AS AN ARGUMENT ABCF000001010011100101110111


# LOOK INTO HAVING THE CHECK ANSWER BUTTON DO DIFFERENT THINGS
# DON'T LIKE THIS IMPLEMENTATION. Find a way to only need one 'on_button_clicked' function

# Process when clicking the "Check" button
# AND
def on_button_clicked(self):
    CheckAnswer(grid1_1, 2, '0001')


# OR
def on_button_clicked2(self):
    CheckAnswer(grid1_2, 2, '0111')


# NOT
def on_button_clicked3(self):
    CheckAnswer(grid1_3, 1, '10')


# NOR
def on_button_clicked4(self):
    CheckAnswer(grid1_4, 2, '1000')


# NAND
def on_button_clicked5(self):
    CheckAnswer(grid1_5, 2, '1110')


# XOR
def on_button_clicked6(self):
    CheckAnswer(grid1_6, 2, '0110')


# XNOR
def on_button_clicked7(self):
    CheckAnswer(grid1_7, 2, '1001')

In [ ]:
#@title Multiple Choice Creation
from IPython.display import IFrame, display, HTML
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout, jslink, IntText, IntSlider
# function of creating button widget
def create_expanded_button(description, button_style, width='400px'):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width=width))

# function of creating grid layout
def create_grid(A,B,C,D):
    grid = GridspecLayout(5, 3, width='470px')
    grid[0,0]=create_expanded_button(A,'primary')
    grid[1,0]=create_expanded_button(B,'primary')
    grid[2,0]=create_expanded_button(C,'primary')
    grid[3,0]=create_expanded_button(D,'primary')  
    grid[4,0]=create_expanded_button('Submit','Danger')  
    return grid


def check_answermc4(self):
  """
  Used with .on_click to makesure the answer is correct
    self: This is a button. We have added 2 new attributes to it
      current_answer: This is the last clicked button and its description value stored as a str
      answer: This is the correct answer from the mcdict

  """
  if self.current_answer == 0:
    return
  if self.answer[0] == self.current_answer[0]:
    self.button_style = "Success"
    self.description = "Correct!"
  else:
    self.button_style = 'Danger'
    self.description = "Wrong, Try Again!"
"""
key: The question number
pair: This should be a list. indexes 0-3 are the choices. 4 is the answer. 5 is the question
answer: This must be the same str value as the choice
question: This is used in the header
"""
mc_dict = {

    "1": ["A. 0101","B. 01010", "C. 1010","D. Syntax Error", "A", "What is the value of A[5:2] where A = 9’b101010101?"],
    "2": ["A. 11010", "B. 10100", "C. 10111", "D. 01000", "B", "What is the value of A << 3 where A  = 5’b11101 ?"],
    "3": ["A. 10111", "B. 11101", "C. 101", "D. Syntax Error", "A", "What is the Value of {A,B} where A = 3’b101 and B = 2’11?"],
    "4": ["A. 1010", "B. 111000", "C. 101010", "D. Syntax Error", "C", "What is the value of {3{2’b10}}?"],
    "5": ["A.logic input [3:0] x", "B. logic [3:0] input x", "C. input logic x [3:0]", "D. input logic [3:0] x", "D", "Which signal is defined correctly?"  ],
    "6": ["A 1’b1", "B. 1’b0", "C. An Error will be thrown", "D. Undefined", "C", "What is the value of x from the code above?"],
    "7": ["A. 1’b1", "B. 1’b0", "C. 2’b11", "D. An Error will be thrown", "A", "What is the value of x from the above code?"] ,
    "8": [ "A. 4'b0001", "B. 4'b0011", "C. 4'b0111", "D. 4'b1001", "D", "What is the value of x in the above code?" ]
}
def multiple_choice4(key, dictionary):
  """
  Creates a multiple choice widget that has 4 options. 
  key: This is the key to the dictionary. It will be a str of digits. 
  dictionary: the dictionary used is mc_dict. Which houses the multiple choice answers.
  grid: The grid that has been set up and can be shown
  header_button: sets the header_button that houses the question description
  """
  answers = dictionary[key]
  question = answers[5]
  A = answers[0]
  B = answers[1]
  C = answers[2]
  D = answers[3]
  grid = create_grid(A,B,C,D)
  correct_answer = answers[4]
  setattr(grid[4,0], "answer", correct_answer)
  current_answer = 0
  setattr(grid[4,0], "current_answer", 0)
  grid[4,0].button_style = "Warning"
  def change_color(self):
    current_btn = int(self.layout.grid_area[-1])
    for i in range(0,4):
      grid[i,0].button_style = 'primary'
    grid[current_btn-1,0].button_style = 'info'
    grid[4,0].current_answer = self.description

  for i in range(0,4):
    grid[i,0].on_click(change_color)
  grid[4,0].on_click(check_answermc4)
  header_button = create_expanded_button(question, 'info', '465px')
  return grid, header_button


In [ ]:
#@title Verilator Install
!apt-get install verilator >/dev/null

In [ ]:
#@title Install PyGithub
!pip3 install PyGithub > /dev/null

In [ ]:
#@title Seven Segment Widget

import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout, jslink, IntText
#Seven Segment Display
def create_expanded_button(description="", button_style='', width='50px', height='50px'):
    return Button(description=description, button_style=button_style, layout=Layout(height=height, width=width))
def create_7_grid():
 grid = GridspecLayout(5,1, width='105px') #Middle Column
 grid[0,0] = create_expanded_button(button_style = "Danger", width = '100px')
 grid[1,0] = create_expanded_button(width = '100px', height = '65px')
 grid [2,0] = create_expanded_button(button_style = "Danger", width = '100px')
 grid [3,0] = create_expanded_button(width = '100px',  height= '65px')
 grid [4,0] = create_expanded_button(button_style = "Danger", width = '100px') 
 return grid

def create_7b_grid(): #Side Columns
 grid = GridspecLayout(2, 1, width='54px')
 grid[0,0] = create_expanded_button(height="150px", width="50px", button_style="Danger")
 grid[1,0] = create_expanded_button(height="150px", width="50px", button_style="Danger")
 return grid

x = create_7b_grid() 
y = create_7_grid()
z= create_7b_grid()
display(widgets.HBox([x,y,z]))

#Value Entry
def cbi(): #Create Bounded Int
  return widgets.BoundedIntText(min = 0, max = 1, layout=Layout(height="auto", width="50px"))
def create_value_grid():
  grid = GridspecLayout(2,8, width = '500px')
  for i in range (7):
    grid[1,i] = cbi()
  i = 0
  for letter in  reversed(['a','b','c','d','e','f','g']):
    grid[0,i] = Button(description=letter, button_style = 'info',layout=Layout(height="auto", width="50px"))
    i += 1
  grid[1,7] = Button(description = "Show", button_style = 'primary',layout=Layout(height="auto", width="100px"))
  return grid
value_list = create_value_grid()
def update_seven(self):
  if(value_list[1,6].value == 1):
    y[0,0].button_style = ''#a
  else:
    y[0,0].button_style = 'Danger'

  #b
  if(value_list[1,5].value == 1):
    z[0,0].button_style = ''
  else:
    z[0,0].button_style = 'Danger'
  #c
  if(value_list[1,4].value == 1):
    z[1,0].button_style = ''
  else:
    z[1,0].button_style = 'Danger'
  #d
  if(value_list[1,3].value == 1):
    y[4,0].button_style = ''
  else:
    y[4,0].button_style = 'Danger'
  #e
  if(value_list[1,2].value == 1):
    x[1,0].button_style = ''
  else:
    x[1,0].button_style = 'Danger'
  #f
  if(value_list[1,1].value == 1):
    x[0,0].button_style = ''
  else:
    x[0,0].button_style = 'Danger'
  #g
  if(value_list[1,0].value == 1):
    y[2,0].button_style = ''
  else:
    y[2,0].button_style = 'Danger'
value_list[1,7].on_click(update_seven)
display(value_list)

GridspecLayout(children=(BoundedIntText(value=0, layout=Layout(grid_area='widget001', height='auto', width='50…

In [ ]:
# @title Install Interactive Simulation
def altair_sim():
  with open("/content/tmp_code/logs/vlt_dump.vcd", "r") as f:
      # Remove Date, Program and  Timescale
      for i in range(20):
          line = f.readline()
          words = line.split()
          try:
            words[0]
          except:
            line = f.readline()
            words = line.split()
            continue
          if words[0] == "$date" or words[0] == "$timescale" or words[0] == "$version":
              for i in range(10):
                  line = f.readline()
                  words = line.split()
                  if words[0] == "$end" or words[-1] == "$end":
                      break
          else:
              break
          line = f.readline()
      # Create the dictionary of lists for inputs
      signals = {}
      line = f.readline()
      words = line.split()
      symbols = {}
      for i in range(50):
          try:
            words[0]
          except:
            line = f.readline()
            words = line.split()
            continue
          if words[0] == "$var":
              signals[words[4]] = []
              symbols[words[3]] = words[4]

          elif words[0] == "$enddefinitions":
              break
          line = f.readline()
          words = line.split()
      k = 0

      value = 0
      for i in range(0, 13000):
          if not words:
              line = f.readline()
              words = line.split()
              continue
          elif words[0] == "$dumpvars":
              line = f.readline()
              words = line.split()
              for i in range(50):
                  if words[0] == "$end":
                      break
                  else:
                      lines = f.readline()
                      words = line.split()

          elif len(words) > 1:
              if words[0][0] == "b":
                  if words[0][1] == "x":
                      value = 0
                  else:
                      try:
                          value = int(words[0][1:-1],2)
                      except:
                          value = int(words[0][1],2)
                  symbol = words[1]
                  signals[symbols[symbol]].append(value)
          elif words[0][0] == "#" and words[0][0] != "0":
              k = k + 1
              for key in signals:
                  if len(signals[key]) < k:
                      try:
                          signals[key].append(signals[key][-1])
                      except:
                          signals[key].append(0)
          else:
              if words[0][0] == "x":
                  value = 0
              else:
                  value = int(words[0][0:-1])
              symbol = words[0][-1]
              signals[symbols[words[0][-1]]].append(value)
          line = f.readline()
          words = line.split()
      length_df = 0
      dataframe = signals
      for i in dataframe:
        if length_df < len(dataframe[i]):
          length_df = len(dataframe[i])
      for i in dataframe:
        if len(dataframe[i]) < length_df:
          dataframe[i].append(dataframe[i][-1])
      # Create a list of variables and their names (ignore types)
      # Create a key to translate the signals
      # Begin Cycling through the signals
  dataframe1 = signals
  df_max = 0
  for i in dataframe:
    if df_max < max(dataframe[i]):
      df_max = max(dataframe[i])
  import altair as alt
  import pandas as pd
  import numpy as np
  from vega_datasets import data
  length_df = 0
  for i in dataframe:
    if len(dataframe[i]) > length_df:
      length_df = len(dataframe[i])
  def make_waveform(dataframe):
    length_df = 0
    for i in dataframe:

      if len(dataframe[i]) > length_df:
        length_df = len(dataframe[i])

    source = pd.DataFrame( dataframe, index=pd.RangeIndex(length_df, name='x'))
    source = source.reset_index().melt('x', var_name='signals', value_name='y')
    scales = alt.selection_interval(bind='scales')

    multi_mouseover = alt.selection_multi(on='mouseover', toggle=True, empty='none')

    selection = alt.selection_multi(fields=['signals'], bind='legend')

    # Create a selection that chooses the nearest point & selects based on x-value
    nearest = alt.selection(type='multi', nearest=True, on='mouseover',
                            fields=['x'], empty='none')

    # The basic line
    line = alt.Chart(source).mark_line(interpolate='step-after').encode(
        x='x:Q',
        y='y:Q',
        color='signals:N'
    )

    # Transparent selectors across the chart. This is what tells us
    # the x-value of the cursor
    selectors = alt.Chart(source).mark_point().encode(
        x='x:Q',
        opacity=alt.value(0),
    ).add_selection(
        nearest,
        scales
    )

    # Draw points on the line, and highlight based on selection
    points = line.mark_point().encode(
        opacity=alt.condition(nearest, alt.value(1), alt.value(0))
    )

    # Draw text labels near the points, and highlight based on selection
    text = line.mark_text(align='left', dx=5, dy=-5).encode(
        text=alt.condition(nearest, 'y:Q', alt.value(''))
    )

    # Draw a rule at the location of the selection
    rules = alt.Chart(source).mark_rule(color='black').encode(
        x='x:Q',
    ).transform_filter(
        nearest
    )

    # Put the five layers into a chart and bind the data
    waveform=alt.layer(
        line, selectors, points, rules, text
    ).encode(
        alt.X(scale=alt.Scale(domain=(0,length_df))),
        alt.Y(scale=alt.Scale(domain=(0,df_max+1))),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
    ).properties(
        width=600, height=400
    ).add_selection(
        selection
    )

    histogram = alt.Chart(source).mark_bar().encode(
      alt.X('y', 
          scale=alt.Scale(domain=(0,df_max + 1))),
      y='signals',
      color='signals:N',

    ).transform_filter(nearest).properties(
        width = 600, height = 200
    )
    text = histogram.mark_text(
        align = 'left',
        baseline = 'middle',
        dx = 3
    ).encode(
        text = 'y:Q'
    )
    histogram = histogram + text  
    return waveform, histogram

  waveform1, histogram1 = make_waveform(dataframe1)
  return waveform1, histogram1



## How the Seven Segment Display Works




### There are in fact seven segments. Imagine that

<br>

![picture](https://raw.githubusercontent.com/byuccl/digital_design_colab/master/SevenSegment/media/display.jpg)  

Seven Segment displays are a commonly used type of display that consist of 7 seperate LED segments that can be turned on in different combinations to produce numbers and letters.

![picture](https://raw.githubusercontent.com/byuccl/digital_design_colab/master/SevenSegment/media/segments.png)  

<br>

Each segment of the display is controlled by a different a signal which will be referred to as segment `a` through segment `g` as shown above. Using different combinations of the seven signals, you can display numbers between `0` and `9`, and letters from `A` to `F`. The 8th signal that gets sent to the display is for the dot.  

<br>

![picture](https://raw.githubusercontent.com/byuccl/digital_design_colab/master/SevenSegment/media/hex_segments.png)  



So how exactly does this work? To keep it simple, because the LED segments share a common cathode, there needs to be a voltage drop from anode to cathode. This means that to turn on a signal, give it a value of `0` and to turn it off give it a value of `1`.

Experiment with the widget below and then fill in all the values for the 16 Hex Values.




In [ ]:
#@title Seven Segment Widget

display(widgets.HBox([x,y,z]))


display(value_list)


GridspecLayout(children=(BoundedIntText(value=0, layout=Layout(grid_area='widget001', height='auto', width='50…

In [ ]:
#@title Seven Segment Decoder
#Key
seven_segment_decode =[
 "1000000",#0
 "1111001",#1
 "0100100",#2
 "0110000",#3
 "0011001",#4
 "0010010",#5
 "0000010",#6
 "1111000",#7
 "0000000",#8
 "0010000",#9
 "0001000",#A
 "0000011",#B
 "1000110", #C
 "0100001",#D
 "0000110", #E
 "0001110"#F
]

def cbi(): #Create Bounded Int
  return widgets.BoundedIntText(min = 0, max = 1, layout=Layout(height="60px", width="75px"))
def create_expanded_button(description="", button_style='info', width='150px', height='50px'):
    return Button(description=description, button_style=button_style, layout=Layout(height=height, width=width))
#first row

def create_expanded_button_primary(description="", button_style='Primary', width='150px', height='50px'):
    return Button(description=description, button_style=button_style, layout=Layout(height=height, width=width))
#first row
first_row = GridspecLayout(1,3, width = '1200px')
first_row[0,0] = create_expanded_button_primary(description = "Hex")
first_row[0,1] = create_expanded_button_primary(description = "Input", width = '320px')
first_row[0,2] = create_expanded_button_primary(description = "Output", width = '710px')

#second row
#first row
second_row = GridspecLayout(1,3, width = '1200px')
second_row[0,0] = create_expanded_button(description = "Character")
second_row[0,1] = create_expanded_button(description = "Binary", width = '320px')
second_row[0,2] = create_expanded_button(description = "Segment [6:0]", width = '710px')
main_block = GridspecLayout(16,13, width = '1200px')

#Third Row

third_row = GridspecLayout(1, 13, width = '1200px')
descr_list = ["data[3]", "data[2]", "data[1]", "data[0]", "CG", "CF", "CE", "CD", "CC", "CB", "CA"]
third_row[0,0] = create_expanded_button(button_style = "info")
third_row[0,12] = create_expanded_button(button_style = "")

i = 1
for l in descr_list:
  third_row[0,i] = create_expanded_button(description = l, width = "75px")
  i += 1

def check_seven(self):
  answer_value = ''
  for i in range(5,12):
    answer_value += str(main_block[self.row, i].value)
  if(answer_value == seven_segment_decode[self.row]):
    print("Correct")
    self.button_style = "Success"
    self.description = "Correct"
  else:
    self.button_style = "Danger"
    self.description = "Incorrect, Try Again"
  print(answer_value + " " + seven_segment_decode[self.row])


#First Column
for i in range(16):
  main_block[i,0] = create_expanded_button(description=str(hex(i))[-1])
#Create a binary pattern
x = 8
for j in range(1,5):
  z = 0
  binary_v = '0'
  for i in range(16):
    z += 1
    main_block[i,j] = create_expanded_button(description = binary_v, button_style = "", width = '75px')
    if (z == x) and binary_v == '1':
      binary_v = '0'
      z = 0
    elif (z == x) and binary_v =='0':
      binary_v = '1'
      z= 0
  x = x/2
for j in range (5,12):
  for i in range (16):
    main_block[i,j] = cbi()
for i in range (16):
  main_block[i,12] = create_expanded_button(description = "Check", button_style = 'Warning')
  setattr(main_block[i,12], 'row', i)
  main_block[i,12].on_click(check_seven)
display(first_row)
display(second_row)
display(third_row)
display(main_block)

GridspecLayout(children=(Button(button_style='primary', description='Hex', layout=Layout(grid_area='widget001'…

GridspecLayout(children=(Button(button_style='info', description='Character', layout=Layout(grid_area='widget0…

GridspecLayout(children=(Button(button_style='info', layout=Layout(grid_area='widget001', height='50px', width…

GridspecLayout(children=(Button(button_style='info', description='0', layout=Layout(grid_area='widget001', hei…

### Driving an Anode

Anodes are what determines which of the 4 Seven Segment displays are lit. If the anode is `0` for a particular panel it will be on. So if `0000` is driven all the panels will be on. 
<br>

![picture](https://raw.githubusercontent.com/byuccl/digital_design_colab/master/SevenSegment/media/cathode.jpg)  

<br>

The problem with the anodes is that they are all driven by the same exact wires to determine what segments are on. As a result they will naturally show the same image 4 times.   

To resolve this, seven segment displays actually oscillate between the four different displays quicker than the human eye can distinguish so that the 4 displays appear to show different images at the same time when in reality they are displayed one at a time.



## The Lab

### Seven Segment Module

In this lab, given a binary value, display the equivalent Hexadecimal character.
So if we input `0000` we want to output `0` and if we input `1100` we want to output `A`.  

| Module Name: |seven_segment|||
| ----------- | ----------- |--|--|
| Port Name      | Direction       |Width|Function|
| data	|Input 	|4| data to display|
|segment 	|Output 	|7 | Cathode signals to the display|


In [ ]:
#@title 
createSimulationWorkSpace("/tmp_code/seven_segment")

/tmp_code/seven_segment


In [ ]:
#@title Verilator TestBench
# !verilator --cc full_add 
!verilator -Wall --trace -cc seven_segment.sv --exe tb_seven_segment.cpp
!make -C obj_dir -f Vseven_segment.mk Vseven_segment > /dev/null
!./obj_dir/Vseven_segment

In [ ]:
#@title Create WaveDrom
df2wd("ss")

In [ ]:
#@title Show WaveDrom
%%wavedrom_magic -h 200 -o /content/tmp_code/ss.txt
---

In [ ]:
#@title Interactive Simulator
waveform1, histogram1 = altair_sim()
waveform1 & histogram1

### The Top Module



| Module Name: |seven_segment_top|||
| ----------- | ----------- |--|--|
| Port Name      | Direction       |Width|Function|
| sw	|Input 	|4| Input to drive seven-segment decoder |
| btnc	|Input 	|1| Will turn on digit point |
| segment	|Output 	|8 | Cathode signals to the display including the point|
| anode	|Output 	|4 | Anode signals for each of the four digits on the display|  

<br>  
This figure shows what is happening in the module. 
<br>  <br>

![picture](https://raw.githubusercontent.com/byuccl/digital_design_colab/master/SevenSegment/media/seven_top.jpg)   



In [ ]:
#@title

createSimulationWorkSpace("/tmp_code/seven_segment_top")

/tmp_code/seven_segment_top


In [ ]:
#@title Interactive Simulator
waveform1, histogram1 = altair_sim()
waveform1 & histogram1


## Compiling with the F4PGA Toolchain


### Installing the Toolchain

In [ ]:
!apt install -y git wget xz-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.13).
wget is already the newest version (1.19.4-1ubuntu2.2).
xz-utils is already the newest version (5.2.2-1.3ubuntu0.1).
xz-utils set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.


In [ ]:
%%bash
git clone https://github.com/chipsalliance/f4pga-examples
cd f4pga-examples
wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O conda_installer.sh


Cloning into 'f4pga-examples'...
--2022-11-14 23:09:16--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2022-11-14 23:09:16--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76607678 (73M) [application/x-sh]
Saving to: ‘conda_installer.sh’

     0K .......... .......... .......... .......... ..........  0% 41.5M 2s
    50K .......... .......... .......... ...

In [ ]:
#Creates the Conda Environment
%%bash
cd f4pga-examples
export F4PGA_INSTALL_DIR=~/opt/f4pga
export FPGA_FAM=xc7
bash conda_installer.sh -u -b -p $F4PGA_INSTALL_DIR/$FPGA_FAM/conda;
source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh";
conda env create -f $FPGA_FAM/environment.yml

PREFIX=/root/opt/f4pga/xc7/conda
Unpacking payload ...
Solving environment: ...working... done

## Package Plan ##

  environment location: /root/opt/f4pga/xc7/conda

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - _openmp_mutex==4.5=1_gnu
    - brotlipy==0.7.0=py39h27cfd23_1003
    - ca-certificates==2022.3.29=h06a4308_1
    - certifi==2021.10.8=py39h06a4308_2
    - cffi==1.15.0=py39hd667e15_1
    - charset-normalizer==2.0.4=pyhd3eb1b0_0
    - colorama==0.4.4=pyhd3eb1b0_0
    - conda-content-trust==0.1.1=pyhd3eb1b0_0
    - conda-package-handling==1.8.1=py39h7f8727e_0
    - conda==4.12.0=py39h06a4308_0
    - cryptography==36.0.0=py39h9ce1e76_0
    - idna==3.3=pyhd3eb1b0_0
    - ld_impl_linux-64==2.35.1=h7274673_9
    - libffi==3.3=he6710b0_2
    - libgcc-ng==9.3.0=h5101ec6_17
    - libgomp==9.3.0=h5101ec6_17
    - libstdcxx-ng==9.3.0=hd4cf53a_17
    - ncurses==6.3=h7f8727e_2
    - openssl==1.1.1n=h7f8727e_0
    - pip==21.2.4=py39h06a4308_0
    - pycosat==0.6.3=py39h27cfd23

                                                                                             

==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 22.9.0

Please update conda by running

    $ conda update -n base -c defaults conda




In [ ]:
#Download F4PGA Arch Defs
%%bash
export F4PGA_INSTALL_DIR=~/opt/f4pga
export FPGA_FAM=xc7
source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh";
conda activate xc7
mkdir -p $F4PGA_INSTALL_DIR/xc7/install


F4PGA_TIMESTAMP='20220907-210059'
F4PGA_HASH='66a976d'
export F4PGA_PACKAGES='install-xc7 xc7a50t_test'

for PKG in $F4PGA_PACKAGES; do
  wget -qO- https://storage.googleapis.com/symbiflow-arch-defs/artifacts/prod/foss-fpga-tools/symbiflow-arch-defs/continuous/install/${F4PGA_TIMESTAMP}/symbiflow-arch-defs-${PKG}-${F4PGA_HASH}.tar.xz | tar -xJC $F4PGA_INSTALL_DIR/${FPGA_FAM}
done

### Compiling with the Toolchain

In [ ]:
#This creates the Makefile
with open("./content/tmp_code/Makefile", "w") as f:
  f.write("""current_dir := ${CURDIR}
TARGET := basys3

TOP := seven_segment_top

XDC := ${current_dir}/*.xdc

SOURCES := $(wildcard ${current_dir}/*.v ${current_dir}/*.sv)

include /content/f4pga-examples/common/common.mk
""")

In [ ]:
#Make the project
%%bash
export F4PGA_INSTALL_DIR=~/opt/f4pga
export FPGA_FAM=xc7
export TARGET=basys3
source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh";
conda activate xc7
cd ./Lab10
make 2> ./error.log 1>./compile.log
cp ./build/basys/*.bit ./codebreaker_serial_top.bit

## Testing it on the board

You can use this configuration file and the program openOCD to download the bitstream to your board. You will need to download this file and the bitstream to your local machine.

In [ ]:
#Create File for openocd
with open("/content/seven_segment_top.cfg", "w") as f:
  f.write("""interface ftdi
ftdi_device_desc "Digilent USB Device"
ftdi_vid_pid 0x0403 0x6010
# channel 1 does not have any functionality
ftdi_channel 0
# just TCK TDI TDO TMS, no reset
ftdi_layout_init 0x0088 0x008b
reset_config none
adapter_khz 10000

source [find cpld/xilinx-xc7.cfg]
source [find cpld/jtagspi.cfg]
init

puts [irscan xc7.tap 0x09]

set xc7a35t "0362D093"
set xc7a100t "13631093"
set code [drscan xc7.tap 32 0]  
puts $code

if { $code == $xc7a35t} {
    puts "The board has an xc7a35t"
}

if { $code == $xc7a100t} {
    puts "The board has an xc7a100t"
}

puts "Programming..."
pld load 0 seven_segment_top.bit
exit"""
)

The following code will download a zip file with the configuration file and bitstream. This will only download if you are using Chrome. If you are using something else, click the file icon on the left, and click the zip file in the content folder to download it.

In [ ]:
!zip seven_segment.zip /content/tmp_code/build/basys3/*.bit /content/seven_segment_top.cfg
import from google.colab import files
files.download('seven_segment.zip')